In [12]:
import pandas as pd
import pickle

In [5]:
def replace_lt_with_previous(article_path):
    path_elements = article_path.split(';')
    for i, element in enumerate(path_elements):
        if element == '<' and i > 1:
            path_elements[i] = path_elements[i-2]
    return ';'.join(path_elements)

In [10]:
file_path = '../data/paths_finished.tsv'
df_finished = pd.read_csv(file_path, sep='\t', header=None)
cols = ['hashedIpAddress', 'timestamp', 'durationInSec', 'path', 'rating']
df_finished.columns = cols
df_finished['target'] = df_finished['path'].apply(lambda x: x.split(';')[-1])
df_finished['type'] = 'finished'
file_path = '../data/paths_unfinished.tsv'
df_unfinished = pd.read_csv(file_path, sep='\t', header=None)
cols = ['hashedIpAddress', 'timestamp', 'durationInSec', 'path', 'target', 'type']
df_unfinished.columns = cols
df = pd.concat([df_finished, df_unfinished], axis=0)
df['start'] = df['path'].apply(lambda x: x.split(';')[0])
df['path_length'] = df['path'].apply(lambda x: len(x.split(';')))
df['path'] = df['path'].apply(replace_lt_with_previous)
# dump to csv
df[['durationInSec', 'path', 'target', 'type', 'start', 'path_length']].to_csv('../data/paths.tsv', sep='\t', index=False)

In [8]:
articles = pd.read_csv('../data/articles.tsv', sep='\t', header=None)
file_path = '../data/links.tsv'
df_links = pd.read_csv(file_path, sep='\t', header=None)
articles.columns = ['article']
degree_out = df_links.groupby(0).count().reset_index()
degree_out.columns = ['article', 'degree_out']
degree_in = df_links.groupby(1).count().reset_index()
degree_in.columns = ['article', 'degree_in']
articles = articles.merge(degree_out, on='article', how='left').fillna(0).merge(degree_in, on='article', how='left').fillna(0)
articles['degree'] = articles['degree_out'] + articles['degree_in']
article_freq = pd.DataFrame(df['path'].apply(lambda x: x.split(';')).explode().value_counts()).reset_index()
articles_cnt = pd.merge( articles, article_freq, left_on='article', right_on='path', how='left').fillna(0)
articles_cnt.head()

,article,degree_out,degree_in,degree,path,count
0,%C3%81ed%C3%A1n_mac_Gabr%C3%A1in,11.0,0.0,11.0,0,0.0
1,%C3%85land,19.0,0.0,19.0,%C3%85land,4.0
2,%C3%89douard_Manet,20.0,0.0,20.0,%C3%89douard_Manet,7.0
3,%C3%89ire,8.0,0.0,8.0,%C3%89ire,13.0
4,%C3%93engus_I_of_the_Picts,10.0,0.0,10.0,0,0.0


In [33]:
with open('tmp/sentiments.pickle', 'rb') as handle:
    sentiments = pickle.load(handle)
    
articles_cnt['sentiment_pos'] = articles_cnt['article'].apply(lambda x: sentiments[x+".txt"][0] if x+".txt" in sentiments else 0)
articles_cnt['sentiment_neu'] = articles_cnt['article'].apply(lambda x: sentiments[x+".txt"][1] if x+".txt" in sentiments else 0)
articles_cnt['sentiment_neg'] = articles_cnt['article'].apply(lambda x: sentiments[x+".txt"][2] if x+".txt" in sentiments else 0)
articles_cnt[['article','degree','sentiment_pos', 'sentiment_neu', 'sentiment_neg']].to_csv('../data/articles_sentiments.tsv', sep='\t', index=False)